# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Weather results into DataFrame

In [2]:
#analysis can be found in WeatherPy.ipynb
city_weather_path = "weather_data/city_weather.csv"
city_weather_df = pd.read_csv(city_weather_path)

### Humidity Heatmap

In [3]:
#adding in gmaps
gmaps.configure(api_key=g_key)
#adding cities locations as latitude and longitude
cities = city_weather_df[["Lat","Long"]]
#setting up the humidity as a number
humidity = city_weather_df["Humidity"].astype(float)

In [4]:
#creating heat map
fig = gmaps.figure()
#set max intensity as max humidity
heatmap_layer = gmaps.heatmap_layer(cities, weights=humidity, dissipating=False, max_intensity=100,point_radius=1.5)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Perfect Weather Cities
Cities that are between 70 to 80 F, low wind (below 10 mph), and no clouds.

In [16]:
best_temp_df = city_weather_df.loc[city_weather_df["Max Temp"] < 78]
best_temp_df2 = best_temp_df.loc[best_temp_df["Max Temp"] > 70]
best_temp_wind_df = best_temp_df2.loc[best_temp_df2["Wind Speed"] < 10]
best_weather_df = best_temp_wind_df.loc[best_temp_wind_df["Cloudiness"] == 0]
count_cities = best_weather_df["City"].count()
print(f'The number of perfect weather cities is {count_cities}.')

The number of perfect weather cities is 10.


### Hotel Map

In [17]:
#added column to the dataframe
best_weather_df["Hotel Name"] =""

#reset dataframe index
best_weather_df = best_weather_df.reset_index(drop=True)

/Users/Madison/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
#base google places API link
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#creating row counter
x = 0

for index, row in best_weather_df.iterrows():

    # variable set up
    target_lat = best_weather_df["Lat"][x]
    target_long = best_weather_df["Long"][x]
    target_coordinates = (f'{target_lat}, {target_long}')
    target_radius = 5000
    target_type = "lodging"


    # parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    #request
    response = requests.get(url, params=params).json()
    results = response["results"]

    #add the first hotel to the dataframe
    best_weather_df.loc[index, 'Hotel Name'] = results[0]['name']
    x = x + 1


In [19]:
#template and added bolding
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
</dl>
"""
#combine with dataframe data
hotel_info = [info_box_template.format(**row) for index, row in best_weather_df.iterrows()]
locations = best_weather_df[["Lat", "Long"]]

In [20]:
#creating new heat map with hotels
fig = gmaps.figure()
#heatmap layer from previous
heatmap_layer = gmaps.heatmap_layer(cities, weights=humidity, dissipating=False, max_intensity=100,point_radius=1.5)
#new hotel layer using hotel_info
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#add layers and display
fig.add_layer(heatmap_layer)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))